# Prompt Tuning 部分

## Step1 导入包

In [ ]:
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer

## Step2 加载数据集

In [ ]:
ds = Dataset.load_from_disk("../resource/alpaca_data_zh/")
ds

## Step3 数据集预处理

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("../resource/Qwen2-7B")
tokenizer

In [ ]:
def process_dataset(example):
    MAX_LENGTH = 256
    input_ids, attention_mask, labels = [], [], []
    instruction = tokenizer("\n".join(["Human: " + example["instruction"], example["input"]]).strip() + "\n\nAssistant: ")
    response = tokenizer(example["output"] + tokenizer.eos_token)
    input_ids = instruction["input_ids"] + response["input_ids"]
    attention_mask = instruction["attention_mask"] + response["attention_mask"]
    labels = [-100] * len(instruction["input_ids"]) + response["input_ids"]
    if len(input_ids) > MAX_LENGTH:
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

In [ ]:
tokenized_ds = ds.map(process_dataset, remove_columns=ds['train'].column_names)
tokenized_ds

In [ ]:
tokenizer.decode(tokenized_ds[1]["input_ids"])

## Step4 创建模型

In [ ]:
model = AutoModelForCausalLM.from_pretrained("../resource/Qwen2-7B")

## Prompt tuning

### PEFT Step1 配置文件

In [ ]:
from peft import PromptTuningConfig, get_peft_model, TaskType, PromptTuningInit

# Soft Prompt
# config = PromptTuningConfig(task_type=TaskType.CAUSAL_LM, num_virtual_tokens=10)
# config
# Hard Prompt
config = PromptTuningConfig(task_type=TaskType.CAUSAL_LM,
                            prompt_tuning_init=PromptTuningInit.TEXT,
                            prompt_tuning_init_text="下面是一段人与智能机器人的对话。",
                            num_virtual_tokens=len(tokenizer("下面是一段人与智能机器人的对话。")["input_ids"]),
                            tokenizer_name_or_path="../resource/Qwen2-7B")
config
# p-tuning
# from peft import PromptEncoderConfig, TaskType, get_peft_model, PromptEncoderReparameterizationType

# config = PromptEncoderConfig(task_type=TaskType.CAUSAL_LM, num_virtual_tokens=10,
#                              encoder_reparameterization_type=PromptEncoderReparameterizationType.MLP,
#                              encoder_dropout=0.1, encoder_num_layers=5, encoder_hidden_size=1024)
# config

### PEFT Step2 创建模型

In [ ]:
model = get_peft_model(model, config)
model

In [ ]:
model.print_trainable_parameters()

## Step5 配置训练参数

In [ ]:
args = TrainingArguments(
    output_dir="./checkpoint",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    logging_steps=10,
    num_train_epochs=1,
    save_steps=500,
)

## Step6 创建训练器

In [ ]:
trainer = Trainer(
    model=model,
    args=args,
    tokenizer=tokenizer,
    train_dataset=tokenized_ds['train'],
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True),
)

## Step7 模型训练

In [ ]:
trainer.train()

## 加载训练好的PEFT模型

In [ ]:
from peft import PeftModel

In [ ]:
model = AutoModelForCausalLM.from_pretrained("../resource/Qwen2-7B")
peft_model = PeftModel.from_pretrained(model=model, model_id="./checkpoint/checkpoint-500/")

## Step8 模型推理

In [ ]:
peft_model = peft_model.cuda()
ipt = tokenizer("Human: {}\n{}".format("考试有哪些技巧？", "").strip() + "\n\nAssistant: ", return_tensors="pt").to(peft_model.device)
print(tokenizer.decode(peft_model.generate(**ipt, max_length=128, do_sample=True)[0], skip_special_tokens=True))